In [2]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# checing if o3d is instaleld
o3d.__version__

'0.17.0'

#### Visualise raw point cloud

In [28]:
pcd = o3d.io.read_point_cloud("./data/P002 2022-01-25 01_24_48.ply")
print(pcd)
print(np.asarray(pcd.points))
# Print min and max coordinates
print("Min:", np.min(pcd.points, axis=0))
print("Max:", np.max(pcd.points, axis=0))

PointCloud with 251759 points.
[[-1.35046351  0.24485803  0.0835193 ]
 [-1.35046351  0.246858    0.0915193 ]
 [-1.35046351  0.24785799  0.0995193 ]
 ...
 [ 1.16753662 -0.51514196  2.53951955]
 [ 1.16803658 -0.52314198  2.53951955]
 [ 1.16803658 -0.531142    2.53951955]]
Min: [-1.47696352 -1.48314226 -0.0274807 ]
Max: [1.18503678 1.22135806 2.57151937]


### Voxel downsampling

- Lidar scans can be very large with millions of points, to improve efficiency, we reduce the overall number of points in the point cloud while retaining the overall structure and shape

In [29]:
voxel_size = 0.05
downpcd = pcd.voxel_down_sample(voxel_size)
o3d.visualization.draw_geometries([downpcd])

### Vertex normal estimation

- `estimate_normals` function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis
    - `radius` = search radius, `max_nn` = maximum nearest neighbour
    - so radius = 0.1 (10cm search radius) and max_nn = 30 (only considers 30 neighbours)
- estimates normal vector (vector perpindicular to tangent plane) for each point in point cloud.
- sensitive to noise and variations & computationally expensive

In [30]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

#### Estimated vertex normal

In [31]:
downpcd.normals[0]

array([ 0.0382553 , -0.07033175, -0.99678984])

In [32]:
print("Normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])

Normal vectors of the first 10 points
[[ 3.82552967e-02 -7.03317526e-02 -9.96789836e-01]
 [ 1.30591400e-01  7.64487856e-02 -9.88484431e-01]
 [ 3.39524397e-04  9.96340889e-01 -8.54676435e-02]
 [-8.59651969e-03 -1.45583988e-02  9.99857066e-01]
 [ 1.53950932e-02 -9.87630672e-02  9.94991883e-01]
 [-1.48672095e-01  9.45758031e-01 -2.88856978e-01]
 [ 9.98957752e-01  4.25496547e-02  1.65207685e-02]
 [-1.21948413e-02 -9.99877159e-01  9.84641230e-03]
 [-3.32356369e-02 -9.96704843e-01 -7.39922200e-02]
 [-2.01307636e-02 -1.09861695e-01  9.93743005e-01]]


### Point cloud distance

- `compute_point_cloud_distance` computes the distance from source to target point cloud (only rly works with 2 sets of PCs)

In [33]:
pcd = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")

In [34]:
dists = pcd.compute_point_cloud_distance(downpcd)
dists = np.asarray(dists)
# print(dists)
ind = np.where(dists > 0.01)[0]
new = pcd.select_by_index(ind)
o3d.visualization.draw_geometries_with_editing([new])

In [35]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

### bounding box

In [36]:
# aabb = downpcd.get_axis_aligned_bounding_box()
# aabb.color = (1, 0, 0)

# # Compute the oriented bounding box
# obb = downpcd.get_oriented_bounding_box()
# obb.color = (0,1,0)  # green

bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(0.5,0.6,0.5), max_bound=(3,4,2))

# Crop the point cloud
cropped_pcd = pcd.crop(bbox)

# Visualize the cropped point cloud
o3d.visualization.draw_geometries([cropped_pcd])

# Display the point cloud with its bounding boxes
# o3d.visualization.draw_geometries([downpcd, aabb, obb])

### dbscan clustering

- grouping point clouds together, density based clustering algorithm
- `cluster_dbscan` takes `eps` = distance to neighbours in cluster, `min_points` = min number of points to form cluster
- the function returns `labels` where the label `-1` indicates noise

In [37]:
import matplotlib.pyplot as plt

In [38]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps = 0.02, min_points=10, print_progress=True))

max_label = labels.max()

print(f"point cloud has {max_label + 1} clusters") # index count starts from 0, so + 1 to get actual number of clusters
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
print(colors)

colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 37
point cloud has 37 clusters
[[0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 ...
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]]


### plane segmentation

- finds plane with largest support in point cloud via `segment_plane`
- `distance_threshold` = max distance a point can have to an estimated plane to be considered an inlier
- `ransac_n` = number of points that are randomly sampled to estimate a plane
- `num_iterations` = how often a random plane is sampled and verified
- Returns a list of indices of the inlier points

In [39]:
model, inliers = pcd.segment_plane(distance_threshold=0.02, ransac_n=5, num_iterations=1000)
[a, b, c, d] = model

print(f"{a}x + {b}y + {c}z + {d} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([outlier_cloud, inlier_cloud])

-0.00574584570168714x + -0.003863946854561101y + 0.9999760272985935z + -0.035509656775515715 = 0
